In [1]:
from pyspark.sql import SparkSession, functions as F

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/15 15:29:05 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 192.168.154.134 instead (on interface eth0)
22/08/15 15:29:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/15 15:29:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/raw/yellow_taxi/')

In [4]:
yellow.show(1, vertical=True, truncate=100)
yellow.printSchema()

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2018-10-01 00:23:34 
 tpep_dropoff_datetime | 2018-10-01 00:44:50 
 passenger_count       | 1.0                 
 trip_distance         | 6.2                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 68                  
 DOLocationID          | 7                   
 payment_type          | 2                   
 fare_amount           | 20.5                
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 21.8                
 congestion_surcharge  | null                
 airport_fee           | null                
only showing top 1 row

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pic

In [ ]:
# create a new column called day_of_week to get day of week for the trip


In [ ]:
# separates the date and time from both the pickup and dropoff date time columns
# creates new columns for pickup/dropoff date and pickup/dropoff time

from pyspark.sql.functions import *
yellow = yellow.withColumn("pickup_date",
                 to_date(col("tpep_pickup_datetime"),"yyyy-MM-dd"))

yellow = yellow.withColumn("dropoff_date",
                 to_date(col("tpep_dropoff_datetime"),"yyyy-MM-dd"))

yellow = yellow.withColumn("pickup_time", date_format('tpep_pickup_datetime', 'HH:mm:ss'))

yellow = yellow.withColumn("dropoff_time", date_format('tpep_dropoff_datetime', 'HH:mm:ss'))

In [ ]:
# Remove columns I deemed irrelevant
# For congestions_surcharge,airport_fee,VendorID and passenger_count I see them as irrelevant due to the fact 
# they don't really contribute to what I am focusing in which is the taxi profits, tipping and location popularity
# For tpep_pickup_datetime and tpep_dropoff_datetime, I created new columns for them.

yellow = yellow.drop('congestion_surcharge','airport_fee','tpep_pickup_datetime','tpep_dropoff_datetime','VendorID'
                     ,'passenger_count', 'store_and_fwd_flag')

yellow_credit = yellow.filter(F.col('payment_type') == 1)


In [ ]:
# remove records that has trips starting before the month of october
yellow_credit = yellow_credit.filter(F.col('pickup_date') >= '2018-10-01')

In [ ]:
# Here we load the curated nba_attendance and convert the Date column into date type
# Then we convert the whole pandas dataframe into a spark dataframe 
import pandas as pd
from pyspark.sql.types import *

nba_attendance = pd.read_csv('../../mast30034-project-1-dustintano10/data/curated/nba_attendance_new.csv')

nba_attendance['Date'] = pd.to_datetime(nba_attendance['Date'], format='%Y%m%d')

schema = StructType([
StructField("Date", DateType(), True),
StructField("Start(ET)", StringType(), True),
StructField("Attend.", StringType(), True),
StructField("Arena", StringType(), True),
StructField("Day_of_week", StringType(), True)
])

nba_attendance_spark = spark.createDataFrame(nba_attendance, schema)

In [ ]:
# joins the nba attendance with the yellow_credit dataframe
yellow_credit = yellow_credit.join(nba_attendance_spark, yellow_credit.pickup_date == nba_attendance_spark.Date, 'left')

In [ ]:
# saves the new yellow_credit dataframe
yellow_credit.write.mode('overwrite').parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')